In [177]:
from mongoengine import connect, disconnect_all
from utils import mongo_to_df
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
import sklearn.model_selection as ms
import numpy as np

### Preparando dados

In [167]:
connect("tcc")
df = mongo_to_df()
disconnect_all()

In [168]:
df.head()

,_id,answer_1,answer_2,answer_3,answer_4,answer_5,answer_6,answer_7,answer_8,answer_9,answer_10,answer_11,evaded,user_id
0,FAKE_answer-01a70f0ff9624093ae94f1eb33829b96,False,True,True,False,True,False,True,False,True,True,False,True,FAKE_user-9c59d8e7f2d243f69150d206d5695733
1,FAKE_answer-8267945f6f21479fa1c58280de074088,False,True,True,True,True,True,False,False,True,True,True,False,FAKE_user-cd43f2f88a654305be0fe4c0f3b75b2a
2,FAKE_answer-b06cbba653a944dea1139b914d8c9013,False,False,False,False,True,True,True,True,True,False,False,False,FAKE_user-b45a2054f5174339a06078ac67ccdbd5
3,FAKE_answer-820e066d68fa416b8aa7ba9e82acdf3b,False,True,True,False,False,False,False,True,True,False,True,True,FAKE_user-0e8adfeacccf400ca1cc9462f40a3b58
4,FAKE_answer-3698a281d1aa4f3fa31f0d5ca9dac0ad,True,True,False,False,True,False,False,False,False,False,True,False,FAKE_user-2774d34e0c79434782ac9f2554a2f1ff


In [169]:
df = df.drop(columns=["_id", "user_id"])

In [170]:
# df = df.astype(int)

In [171]:
df.head()

,answer_1,answer_2,answer_3,answer_4,answer_5,answer_6,answer_7,answer_8,answer_9,answer_10,answer_11,evaded
0,False,True,True,False,True,False,True,False,True,True,False,True
1,False,True,True,True,True,True,False,False,True,True,True,False
2,False,False,False,False,True,True,True,True,True,False,False,False
3,False,True,True,False,False,False,False,True,True,False,True,True
4,True,True,False,False,True,False,False,False,False,False,True,False


In [172]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   answer_1   100 non-null    bool 
 1   answer_2   100 non-null    bool 
 2   answer_3   100 non-null    bool 
 3   answer_4   100 non-null    bool 
 4   answer_5   100 non-null    bool 
 5   answer_6   100 non-null    bool 
 6   answer_7   100 non-null    bool 
 7   answer_8   100 non-null    bool 
 8   answer_9   100 non-null    bool 
 9   answer_10  100 non-null    bool 
 10  answer_11  100 non-null    bool 
 11  evaded     100 non-null    bool 
dtypes: bool(12)
memory usage: 1.3 KB


In [173]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

### Separação dos dados

In [215]:
import sklearn
sklearn.random.seed(21)

In [216]:
# X_train, X_test, y_train, y_test = ms.train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_test, y_train, y_test = ms.train_test_split(X, y, test_size=0.2, shuffle=False, random_state=1)

In [217]:
len(X_train)

80

### Treinando modelo

In [218]:
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

LogisticRegression()

### Previsão

In [219]:
y_pred = classifier.predict(X_test)

### Matriz de confusão

In [220]:
cm = confusion_matrix(y_test, y_pred)

Dos valores que eram 0, quantos que realmente deram 0? 8 (`cm[0][0]`).

Dos valores que eram 0, quantos que deram 1? 2 (`cm[0][1]`).

Dos valores que eram 1, quantos que deram 0? 5 (`cm[1][0]`).

Dos valores que eram 1, quantos que realmente deram 1? 5 (`cm[1][1]`).

In [221]:
cm

array([[3, 6],
       [4, 7]])

In [222]:
y_result = np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1)

In [223]:
# y_result

In [224]:
print(accuracy_score(y_test, y_pred))

0.5
